In [84]:
import pandas as pd
import numpy as np
import csv
from matplotlib import pyplot as plt
from scipy.linalg import fractional_matrix_power
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from sklearn.cluster import KMeans
from sklearn.manifold import SpectralEmbedding
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
# import sklearn import 
N = 10000
n = 6000

In [80]:
# feature 10000*1084
feature = np.genfromtxt('Extracted_features.csv',delimiter = ',',dtype = 'float')
# seed 60*2
seed = np.genfromtxt('Seed.csv',delimiter = ',',dtype = 'int')
# adjacency matrix A
A = np.zeros((n,n))
with open('Graph.csv','rt') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        A[int(row[0])-1,int(row[1])-1] = 1
        A[int(row[1])-1,int(row[0])-1] = 1

In [81]:
degree = np.sum(A,axis=0)
D_nhalf = fractional_matrix_power(np.diag((degree)),-0.5)
# normalized laplacian matrix L
Ln = np.identity(n) - D_nhalf.dot(A).dot(D_nhalf)
# eigenvectors of L
W,V = np.linalg.eig(Ln)
W

array([  1.22082156e-16,   1.76196144e-01,   2.79038424e-01, ...,
         9.99936995e-01,   9.99938445e-01,   9.99995980e-01])

In [78]:
V = V[:,1:]

In [82]:
# pca
pca = PCA(n_components = 2,random_state = 4)
X = pca.fit_transform(feature)
# kmeans
# initialize kmeans with mean of seeds
ip = np.vstack(np.mean(X[seed[seed[:,1] == i,0] - 1,:], axis = 0) for i in range(10))
# kmeans
kmeans = KMeans(n_clusters = 10, init = ip).fit(X)

In [87]:
# visual
plt.figure(figsize=(6, 4))
for i in range(N):
    plt.text(X[i, 0], X[i, 1], str(kmeans.labels_[i]),
             color=str(kmeans.labels_[i] / 10.,1),
             fontdict={'weight': 'bold', 'size': 9})

plt.xticks([])
plt.yticks([])
plt.title('PCA+Kmeans of 10000 points', size=17)
plt.axis('off')
plt.tight_layout()
plt.show()

ValueError: Invalid RGBA argument: 0.4

In [ ]:
accmatrix = np.zeros((100,100,100))
accrfmatrix = np.zeros((100,100,100))
for n1 in range(10,100,10):
    for n2 in range(10,100,10):
        for n3 in range(10,min(n1,n2),10):
            try:
                temp = np.zeros(3)
                temp2 = np.zeros(3)
                for i in range(3):
                    # spectral embedding
                    X1 = V[:,0:n1]
                    # PCA
                    pca = PCA(n_components = n2)
                    X2 = pca.fit_transform(feature[0:n,:])
                    # CCA
                    cca = CCA(n_components = n3)
                    cca.fit(X1,X2)
                    X1_c,X2_c = cca.transform(X1,X2)
                    # Kmeans
                    # initialize kmeans with mean of seeds
                    ip = np.vstack(np.mean(X2_c[seed[seed[:,1] == i,0] - 1,:], axis = 0) for i in range(10))
                    # kmeans
                    kmeans = KMeans(n_clusters = 10, init = ip, random_state = 0).fit(X2_c)
                    # kmeans result of seeds
                    r1 = np.hstack((seed, np.reshape(kmeans.labels_[seed[:,0] - 1],(len(seed),1))))
                    temp[i] = sum(r1[:,1] == r1[:,2])/(1.0*len(r1))
                    # cross validation with random forest
                    rf = RandomForestClassifier(n_estimators = 100, max_features='auto')
                    temp2[i] = np.mean(cross_val_score(rf, feature[0:n,:], kmeans.labels_, cv=10))
                accmatrix[n1,n2,n3] = np.mean(temp)
                accrfmatrix[n1,n2,n3] = np.mean(temp2)
            except:
                print(n1,n2,n3)
np.save('accmatrix_1',accmatrix)
np.save('accrfmatrix_1',accrfmatrix)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one

30 60 20


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one

In [29]:
accsort = np.sort(accmatrix,axis = None)
accsort = accsort[::-1]
accsort

array([ 0.74444444,  0.74444444,  0.74444444, ...,  0.        ,
        0.        ,  0.        ])

In [36]:
for i in range(10):
    print (np.where(accmatrix == accsort[i]))

(array([30, 30, 60, 60], dtype=int64), array([60, 90, 70, 90], dtype=int64), array([10, 10, 10, 20], dtype=int64))
(array([30, 30, 60, 60], dtype=int64), array([60, 90, 70, 90], dtype=int64), array([10, 10, 10, 20], dtype=int64))
(array([30, 30, 60, 60], dtype=int64), array([60, 90, 70, 90], dtype=int64), array([10, 10, 10, 20], dtype=int64))
(array([30, 30, 60, 60], dtype=int64), array([60, 90, 70, 90], dtype=int64), array([10, 10, 10, 20], dtype=int64))
(array([70], dtype=int64), array([90], dtype=int64), array([20], dtype=int64))
(array([60, 70, 80], dtype=int64), array([80, 70, 80], dtype=int64), array([10, 10, 50], dtype=int64))
(array([60, 70, 80], dtype=int64), array([80, 70, 80], dtype=int64), array([10, 10, 50], dtype=int64))
(array([60, 70, 80], dtype=int64), array([80, 70, 80], dtype=int64), array([10, 10, 50], dtype=int64))
(array([40], dtype=int64), array([80], dtype=int64), array([10], dtype=int64))
(array([60, 60, 70], dtype=int64), array([70, 80, 70], dtype=int64), arra

In [68]:
n1 = 30
n2 = 60
n3 = 10
# spectral embedding
X1 = V[:,0:n1]
# PCA
pca = PCA(n_components = n2,random_state = 4)
X2 = pca.fit_transform(feature[0:n,:])
# CCA
cca = CCA(n_components = n3)
cca.fit(X1,X2)
X1_c,X2_c = cca.transform(X1,X2)
# Kmeans
# initialize kmeans with mean of seeds
ip = np.vstack(np.mean(X2_c[seed[seed[:,1] == i,0] - 1,:], axis = 0) for i in range(10))
# kmeans
kmeans = KMeans(n_clusters = 10, init = ip).fit(X2_c)
# kmeans result of seeds
r1 = np.hstack((seed, np.reshape(kmeans.labels_[seed[:,0] - 1],(len(seed),1))))
acc = sum(r1[:,1] == r1[:,2])/(1.0*len(r1))
confm = np.zeros((10,10),dtype = int)
for i in range(len(r1)):
    confm[r1[i,1],r1[i,2]]+=1
acc

0.73333333333333328

In [ ]:
print '\n'.join(' '.join(str(cell) for cell in row) for row in confm)

In [ ]:
# try GMM
n1 = 30
n2 = 60
n3 = 10
# spectral embedding
X1 = V[:,0:n1]
# PCA
pca = PCA(n_components = n2,random_state = 4)
X2 = pca.fit_transform(feature[0:n,:])
# CCA
cca = CCA(n_components = n3)
cca.fit(X1,X2)
X1_c,X2_c = cca.transform(X1,X2)
# GMM
# initialize kmeans with mean of seeds
ip = np.vstack(np.mean(X2_c[seed[seed[:,1] == i,0] - 1,:], axis = 0) for i in range(10))
# GMM
gmm = GaussianMixture(n_components = 10,means_init = ip).fit(X1_c).predict(X1_c)
# GMM result of seeds
r1 = np.hstack((seed, np.reshape(gmm[seed[:,0] - 1],(len(seed),1))))
acc = sum(r1[:,1] == r1[:,2])/(1.0*len(r1))
confm = np.zeros((10,10),dtype = int)
for i in range(len(r1)):
    confm[r1[i,1],r1[i,2]]+=1
acc

In [69]:
print '\n'.join(' '.join(str(cell) for cell in row) for row in confm)

5 0 0 0 0 0 1 0 0 0
0 6 0 0 0 0 0 0 0 0
0 0 5 0 1 0 0 0 0 0
0 0 0 4 0 1 0 0 1 0
0 0 0 0 3 1 0 0 0 2
0 0 0 0 0 5 0 0 0 1
1 0 0 0 1 0 4 0 0 0
0 0 0 0 1 0 0 4 0 1
0 0 0 0 1 1 0 0 3 1
0 0 0 0 1 0 0 0 0 5


In [ ]:
# build random forest using 6000 samples, select parameters using cross validation
for i in [0.3,0.6,0.9]:
    rf = RandomForestClassifier(n_estimators = 500,max_features=i)
    print (np.mean(cross_val_score(rf, feature[0:n,:], kmeans.labels_, cv=5)))

0.556312501982
0.54462783712


In [76]:
# predict 4000 samples, save to txt
rf.fit(feature[0:n,:],kmeans.labels_)
result = rf.predict(feature[n:,:])
r = np.hstack((np.reshape([i+n+1 for i in range(N-n)],(N-n,1)), np.reshape(result,(N-n,1))))
np.savetxt('result2.csv', r, delimiter = ',', fmt = '%d', header = "Id,Label", comments = '')